In [1]:
from qcnn.quantum.operation import C2Q
import pennylane as qml
from pennylane import numpy as np

device = qml.device('default.qubit', wires=range(3))

theta = np.ones(7)
phi = np.ones(7)
t = np.ones(7)

@qml.qnode(device)
def circuit():
    C2Q(theta, phi, t, wires=range(3), angles=True)
    return qml.state()
    
circuit()

tensor([ 0.67587122-9.13476639e-18j,  0.19949589+3.10696442e-01j,
         0.19949589+3.10696442e-01j, -0.08394154+1.83415602e-01j,
         0.19949589+3.10696442e-01j, -0.08394154+1.83415602e-01j,
        -0.08394154+1.83415602e-01j, -0.10909263+1.55507768e-02j], requires_grad=True)